In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
import scipy.stats as stats

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy 

import scipy.interpolate as interpolate
import scipy.constants as constants

In [2]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [3]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

### Setup parameters

In [4]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [5]:
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [6]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [7]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)

***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  175.2406092149549
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  392.6832937186789
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  233.0708308067062
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  341.818416019317
***************************************************
Z2-amplitude:  2.81e+02
reduced chi2:  320.1151243984675
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  235.2380260183528
***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  250.1490478570203
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  270.7275591716337
***************************************************
Z2-amplitude:  2.67e+02
reduced chi2:  318.6342737410096
********************

In [8]:
print("check if scale is positive: ", gfw.scale_Z2)
# gfw.scale_Z2 *= -1

check if scale is positive:  1.8474324693e+17


In [9]:
def get_Yoffset(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return (437.3 - 25 * 9.5) - (aux.voltage_to_position(np.mean(bb.cant_pos[1])))
def get_Stroke(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return -np.std(aux.voltage_to_position(bb.cant_pos[1]))*np.sqrt(2)*2
def get_Dist(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return 392.2 - aux.voltage_to_position(np.mean(bb.cant_pos[0]))

In [10]:
z_sep = np.mean(height_cal)
y_offset = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Yoffset)(i) for i in tqdm(range(0, 10000, 100)))))
stroke = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Stroke)(i) for i in tqdm(range(0, 10000, 100)))))
dist = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Dist)(i) for i in tqdm(range(0, 10000, 100)))))
G_lambda = float(10)
print(z_sep, y_offset, stroke, dist, G_lambda)

100%|██████████| 100/100 [00:00<00:00, 892.67it/s]


-15.859100393 4.88214369679 -202.765748131 13.9887662292 10.0


In [11]:
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])

Loading Gravity Data... Done!
81 9.74070978211e-06
Loaded Yukawa Force


In [12]:
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = np.sqrt(2)*fft_norm(5000, 5000)

_,ax = plt.subplots()
x = shaking378_template
fft = np.abs(np.fft.rfft(x))*scale*1.85/1.55
freq = np.fft.rfftfreq(len(x), d=1./5000)
ax.semilogy(freq, fft, label='7.6 $\mu m$ sphere - Z force')
ax.set(xlim=(1,60), ylim=(None,None), xlabel='frequency [Hz]', ylabel='Force [N]', title=r'$\alpha=1x10^{8}$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
def func_filter(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20201211/Bead1/InitialTest/Data'+str(18+i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    res = bb.z2
    return (res)

In [14]:
test_phases = np.array(Parallel(n_jobs=46)(delayed(func_filter)(i) for i in tqdm(range(10000))))

100%|██████████| 10000/10000 [01:15<00:00, 132.41it/s]


In [15]:
def func_sideband(x, gravity_freqs_all):
    res = []
    for freq_ in gravity_freqs_all:
        b, a = signal.butter(3, [2.*(freq_+1.5-bandwidth/2.)/5000, 2.*(freq_+1.5+bandwidth/2.)/5000], btype = 'bandpass')
        responsefilt = signal.filtfilt(b, a, x)
        res.append(np.std(responsefilt))
    return np.array(res)

In [27]:
ll = likelihood_analyser.LikelihoodAnalyser()

def get_alpha(i):

    data_z2 = np.mean(test_phases[i:(i+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, np.pi],
              'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
#     noises_std = np.array([1])
    noises_std = func_sideband(data_z2, gravity_freqs_all)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    
#     PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
#     PL_array -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PL_array))
        
    return mm_tmp.values[0]

In [55]:
A_array = np.arange(-5,5,0.1)


In [56]:
PLs_all_noise = np.array(Parallel(n_jobs=46)(delayed(get_alpha)(i) for i in tqdm(range(1000))))





  0%|          | 0/1000 [00:00<?, ?it/s]



  5%|▍         | 46/1000 [00:01<00:23, 39.82it/s]



  5%|▍         | 46/1000 [00:16<00:23, 39.82it/s]



  9%|▉         | 92/1000 [01:17<07:50,  1.93it/s]



 14%|█▍        | 138/1000 [02:34<12:23,  1.16it/s]



 18%|█▊        | 184/1000 [03:50<14:55,  1.10s/it]



 23%|██▎       | 230/1000 [05:05<16:11,  1.26s/it]



 28%|██▊       | 276/1000 [06:21<16:37,  1.38s/it]



 32%|███▏      | 322/1000 [06:43<12:29,  1.11s/it]

KeyboardInterrupt: 

In [28]:
PLtmp = []

In [33]:
A_array = np.arange(-1,1,0.01)
PLtmp.append(get_alpha(4))

MLEs:  <ValueView of Minuit at 7f9a800076e8>
  A: 0.0030165706848492846
  phi: 0.0
  sigma: 2.6676882882714636


In [68]:
PLs_summed_dig = []
PLscopy = copy.deepcopy(PLs_all_noise[0])
PLs_summed_dig = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_dig += pp_
PLs_summed_dig -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_dig))

In [69]:
_,ax = plt.subplots()
ax.plot(A_array*100, PLs_summed_dig, '*-')
ax.plot(A_array*100, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(-10,10), xlabel=r'$\alpha \, [10^{6}]$', ylabel = 'Profile-Likelihood', title='Digitization noise files ')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0),
 (-10.0, 10.0),
 Text(0.5, 0, '$\\alpha \\, [10^{6}]$'),
 Text(0, 0.5, 'Profile-Likelihood'),
 Text(0.5, 1.0, 'Digitization noise files ')]

In [15]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(1,13)[1:]
# gravity_freqs_all = np.array([36])
G_lambda = 10
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])


fft_angles = np.angle(np.fft.rfft(shaking378_template))
fft_fft = np.abs(np.fft.rfft(shaking378_template)) * 2 / np.sqrt(5000 * 5000)
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

gravity_phases = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    
    gravity_phases.append(phase_all)

gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
# gravity_phases = np.vstack(gravity_phases).T

Loading Gravity Data... Done!
81 9.74070978211e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data


In [41]:
mm_gravity = get_alpha(0)

MLEs:  <ValueView of Minuit at a6db948>
  A: 1.554781157465186
  phi: 0.0
  sigma: 812.7864802703888


### EDM settings

In [16]:
base_path = r'/home/nadavp/tmp/'
fname = 'new_trap_attractor_shield_potential_efield.txt'


positions_to_sample = np.array([
                                [13.9e-6, 0.0e-6, -15.23e-6], \
                               ])

dipole_units = constants.elementary_charge * 1.0e-6
dipole_moment = np.array([0.0, 0.0, 100.0]) * dipole_units


In [17]:
#############################################################
######  LOAD DATA FROM COMSOL OUTPUT TO NUMPY ARRAYS  #######
#############################################################

path = os.path.join(base_path, fname)

with open(path, 'r') as f:
    lines  =  f.readlines()

for lineind, line in enumerate(lines):
    if 'Grid' in line:
        grid_ind = lineind + 1
    elif 'V (V)' in line:
        voltage_ind = lineind + 1
    elif 'es.Ex (V/m)' in line:
        ex_ind = lineind + 1
    elif 'es.Ey (V/m)' in line:
        ey_ind = lineind + 1
    elif 'es.Ez (V/m)' in line:
        ez_ind = lineind + 1

xvec_str = lines[grid_ind].strip('\n')
yvec_str = lines[grid_ind+1].strip('\n')
zvec_str = lines[grid_ind+2].strip('\n')

xvec = np.array( [float(val) for val in xvec_str.split()] )
yvec = np.array( [float(val) for val in yvec_str.split()] )
zvec = np.array( [float(val) for val in zvec_str.split()] )

lenx = len(xvec)
leny = len(yvec)
lenz = len(zvec)

voltage = np.zeros((lenx, leny, lenz))
efieldx = np.zeros((lenx, leny, lenz))
efieldy = np.zeros((lenx, leny, lenz))
efieldz = np.zeros((lenx, leny, lenz))

for j in range(leny):
    for k in range(lenz):
        voltage_line = lines[voltage_ind + j + k*leny]
        voltage[:,j,k] = np.array( [float(val) for val in voltage_line.strip('\n').split()] ) 

        efieldx_line = lines[ex_ind + j + k*leny]
        efieldx[:,j,k] = np.array( [float(val) for val in efieldx_line.strip('\n').split()] )

        efieldy_line = lines[ey_ind + j + k*leny]
        efieldy[:,j,k] = np.array( [float(val) for val in efieldy_line.strip('\n').split()] )

        efieldz_line = lines[ez_ind + j + k*leny]
        efieldz[:,j,k] = np.array( [float(val) for val in efieldz_line.strip('\n').split()] )





#############################################################
##############  BUILD 3D GRID INTERPOLATORS  ################
#############################################################

voltage_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), voltage)

efieldx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldx)
efieldy_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldy)
efieldz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldz)

In [90]:
dipole_moment = np.array([0.0, 0.0, 100.0]) * dipole_units
x_att = [100*np.sin(3*t/5000*2*np.pi) for t in range(5000)]
positions_to_sample4 = np.array([[13.9*1e-6, (offset_-4.9)*1e-6, -15.23e-6] for offset_ in x_att])

In [91]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
force_x6 = []
force_z6 = []
for point in positions_to_sample4:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()
    force_x6.append(force_on_dipole[0])
    force_z6.append(force_on_dipole[2])


    MS dipole [e um] :  [   0.    0.  100.]

    MS Postion [um]  :  [ 13.9   -4.9  -15.23]
        Efield [V/m] :  [-3629.04376377   -32.38996578   745.93786973]
Force on dipole [N]  :  [ -2.84274175e-15  -6.11284832e-17  -1.00784708e-15]

    MS Postion [um]  :  [ 13.9         -4.52300977 -15.23      ]
        Efield [V/m] :  [-3624.9863892    -32.05054669   744.59604836]
Force on dipole [N]  :  [ -2.84163885e-15  -5.17088092e-17  -1.02290608e-15]

    MS Postion [um]  :  [ 13.9         -4.14602491 -15.23      ]
        Efield [V/m] :  [-3620.9290723    -31.71113242   743.25424606]
Force on dipole [N]  :  [ -2.84053597e-15  -4.22892692e-17  -1.03796488e-15]

    MS Postion [um]  :  [ 13.9         -3.76905076 -15.23      ]
        Efield [V/m] :  [-3617.66080821   -31.09434368   742.196489  ]
Force on dipole [N]  :  [ -2.83193349e-15  -3.34813872e-17  -1.05295370e-15]

    MS Postion [um]  :  [ 13.9         -3.39209268 -15.23      ]
        Efield [V/m] :  [-3614.89149269   -30.30220

    MS Postion [um]  :  [ 13.9          9.74974012 -15.23      ]
        Efield [V/m] :  [-3583.26005986   -18.21376774   827.73170807]
Force on dipole [N]  :  [ -3.39782895e-15   3.14363021e-16  -6.63300527e-16]

    MS Postion [um]  :  [ 13.9         10.12255891 -15.23      ]
        Efield [V/m] :  [-3586.5531662     -5.99602911   846.31123839]
Force on dipole [N]  :  [ -3.47885282e-15   2.76579018e-16  -6.45377408e-16]

    MS Postion [um]  :  [ 13.9        10.4951642 -15.23     ]
        Efield [V/m] :  [ -3.58882603e+03   2.52000541e-01   8.39447967e+02]
Force on dipole [N]  :  [ -3.49951168e-15   1.04195664e-16  -6.42999341e-16]

    MS Postion [um]  :  [ 13.9        10.8675507 -15.23     ]
        Efield [V/m] :  [-3591.09755201     6.49636125   832.58872544]
Force on dipole [N]  :  [ -3.52015840e-15  -6.80864639e-17  -6.40622671e-16]

    MS Postion [um]  :  [ 13.9        11.2397131 -15.23     ]
        Efield [V/m] :  [-3597.70576238     4.59991923   832.07039708]
Force on di

    MS Postion [um]  :  [ 13.9         23.72235511 -15.23      ]
        Efield [V/m] :  [-3648.07720146  -111.17395508   820.37673973]
Force on dipole [N]  :  [ -3.37179334e-15  -1.31396024e-16  -2.17039249e-15]

    MS Postion [um]  :  [ 13.9         24.08336975 -15.23      ]
        Efield [V/m] :  [-3642.91810847  -117.09176788   820.20427539]
Force on dipole [N]  :  [ -3.48271214e-15   2.33363904e-16  -1.86712104e-15]

    MS Postion [um]  :  [ 13.9         24.44397248 -15.23      ]
        Efield [V/m] :  [-3636.45440995  -104.49503737   817.56397591]
Force on dipole [N]  :  [ -3.61796176e-15   5.55907461e-17  -1.48970928e-15]

    MS Postion [um]  :  [ 13.9         24.80415816 -15.23      ]
        Efield [V/m] :  [-3629.99818681   -91.91287518   814.92672997]
Force on dipole [N]  :  [ -3.75305496e-15  -1.21976815e-16  -1.11273400e-15]

    MS Postion [um]  :  [ 13.9         25.16392168 -15.23      ]
        Efield [V/m] :  [-3626.48745502   -80.92208367   815.85578881]
Force on

    MS Postion [um]  :  [ 13.9         37.79159943 -15.23      ]
        Efield [V/m] :  [-3685.00225337    59.07854015   888.44541787]
Force on dipole [N]  :  [ -3.35142486e-15  -2.48108613e-16  -1.30877502e-15]

    MS Postion [um]  :  [ 13.9         38.13220503 -15.23      ]
        Efield [V/m] :  [-3675.50136294    51.61400712   877.23984721]
Force on dipole [N]  :  [ -3.34221185e-15  -4.01824725e-16  -1.37413199e-15]

    MS Postion [um]  :  [ 13.9         38.47219906 -15.23      ]
        Efield [V/m] :  [-3664.44958598    29.42348459   851.43059077]
Force on dipole [N]  :  [ -3.34312535e-15  -6.23863076e-16  -1.57398850e-15]

    MS Postion [um]  :  [ 13.9         38.81157667 -15.23      ]
        Efield [V/m] :  [-3653.41784609     7.27319385   825.66812695]
Force on dipole [N]  :  [ -3.34403718e-15  -8.45498868e-16  -1.77348266e-15]

    MS Postion [um]  :  [ 13.9         39.15033303 -15.23      ]
        Efield [V/m] :  [-3647.90454305    -8.69756183   808.41800724]
Force on

    MS Postion [um]  :  [ 13.9         50.89189341 -15.23      ]
        Efield [V/m] :  [-3660.05823652    22.28587145   853.21493676]
Force on dipole [N]  :  [ -2.98367366e-15  -8.62660790e-17  -2.78512726e-17]

    MS Postion [um]  :  [ 13.9         51.20435942 -15.23      ]
        Efield [V/m] :  [-3662.28323986    23.58378868   853.14856797]
Force on dipole [N]  :  [ -2.96555608e-15  -6.54821167e-17  -6.27935073e-17]

    MS Postion [um]  :  [ 13.9         51.51602807 -15.23      ]
        Efield [V/m] :  [-3664.46327988    24.83234605   853.07943779]
Force on dipole [N]  :  [ -2.94446761e-15  -3.83500812e-17  -1.20682607e-16]

    MS Postion [um]  :  [ 13.9         51.82689491 -15.23      ]
        Efield [V/m] :  [-3666.63771155    26.0776914    853.01048547]
Force on dipole [N]  :  [ -2.92343340e-15  -1.12878455e-17  -1.78422781e-16]

    MS Postion [um]  :  [ 13.9         52.13695554 -15.23      ]
        Efield [V/m] :  [-3668.77983844    26.64008325   853.15275924]
Force on

...

    MS Postion [um]  :  [ 13.9        -63.47501166 -15.23      ]
        Efield [V/m] :  [-3526.04034787  -119.10731262   619.41317134]
Force on dipole [N]  :  [ -2.93897519e-15  -1.08164033e-16  -1.94962092e-15]

    MS Postion [um]  :  [ 13.9        -63.16904797 -15.23      ]
        Efield [V/m] :  [-3519.34143352  -124.92279145   613.88533063]
Force on dipole [N]  :  [ -2.98464717e-15  -7.15374937e-17  -1.92408914e-15]

    MS Postion [um]  :  [ 13.9        -62.86225614 -15.23      ]
        Efield [V/m] :  [-3516.43725974  -126.63952622   612.30683085]
Force on dipole [N]  :  [ -3.04844261e-15  -9.75860015e-18  -1.86997217e-15]

    MS Postion [um]  :  [ 13.9        -62.55464055 -15.23      ]
        Efield [V/m] :  [-3518.2172503   -123.29775656   615.60239934]
Force on dipole [N]  :  [ -3.13455920e-15   8.30156310e-17  -1.78061917e-15]

    MS Postion [um]  :  [ 13.9        -62.24620556 -15.23      ]
        Efield [V/m] :  [-3520.00198224  -119.94708538   618.90674628]
Force on

    MS Postion [um]  :  [ 13.9        -50.63462811 -15.23      ]
        Efield [V/m] :  [-3595.06676729   -65.70423677   720.76334481]
Force on dipole [N]  :  [ -2.79964716e-15   3.72211575e-19  -9.97968745e-16]

    MS Postion [um]  :  [ 13.9        -50.29904997 -15.23      ]
        Efield [V/m] :  [-3595.36977691   -65.44274442   718.53336594]
Force on dipole [N]  :  [ -2.72064399e-15  -4.34646211e-17  -8.10031052e-16]

    MS Postion [um]  :  [ 13.9        -49.96282661 -15.23      ]
        Efield [V/m] :  [-3595.42607331   -65.11850965   716.41081283]
Force on dipole [N]  :  [ -2.64764643e-15  -7.82987182e-17  -6.40242350e-16]

    MS Postion [um]  :  [ 13.9        -49.62596281 -15.23      ]
        Efield [V/m] :  [-3593.48925862   -64.29200225   715.18463289]
Force on dipole [N]  :  [ -2.62413963e-15  -3.99572605e-17  -6.19324632e-16]

    MS Postion [um]  :  [ 13.9        -49.28846335 -15.23      ]
        Efield [V/m] :  [-3591.5487892    -63.46393525   713.95613916]
Force on

    MS Postion [um]  :  [ 13.9       -36.7562281 -15.23     ]
        Efield [V/m] :  [-3560.94464396    -5.97087865   735.85701439]
Force on dipole [N]  :  [ -3.08339152e-15  -2.95507809e-18  -1.89478683e-15]

    MS Postion [um]  :  [ 13.9        -36.39865197 -15.23      ]
        Efield [V/m] :  [-3564.93729219    -7.24390265   735.36477747]
Force on dipole [N]  :  [ -3.09409016e-15  -1.40429273e-17  -1.91296571e-15]

    MS Postion [um]  :  [ 13.9        -36.04062816 -15.23      ]
        Efield [V/m] :  [-3568.934939      -8.5185204    734.87192429]
Force on dipole [N]  :  [ -3.10480218e-15  -2.51446580e-17  -1.93116735e-15]

    MS Postion [um]  :  [ 13.9        -35.68216178 -15.23      ]
        Efield [V/m] :  [-3572.24812528   -13.4385929    735.81919232]
Force on dipole [N]  :  [ -3.13987367e-15   9.54053638e-17  -1.98746898e-15]

    MS Postion [um]  :  [ 13.9        -35.32325792 -15.23      ]
        Efield [V/m] :  [-3575.4771236    -18.83102295   736.95200478]
Force on di

    MS Postion [um]  :  [ 13.9        -22.15480287 -15.23      ]
        Efield [V/m] :  [-3584.07783164   -31.56474468   752.59877079]
Force on dipole [N]  :  [ -3.49914991e-15  -2.36531925e-16  -7.95967727e-16]

    MS Postion [um]  :  [ 13.9        -21.78334447 -15.23      ]
        Efield [V/m] :  [-3573.25585227   -25.30085915   744.15529388]
Force on dipole [N]  :  [ -3.44283906e-15  -2.77777925e-16  -7.68861784e-16]

    MS Postion [um]  :  [ 13.9        -21.41164612 -15.23      ]
        Efield [V/m] :  [-3575.45155102   -24.26004126   744.30404519]
Force on dipole [N]  :  [ -3.47690948e-15  -3.48474267e-16  -7.38065662e-16]

    MS Postion [um]  :  [ 13.9       -21.0397131 -15.23     ]
        Efield [V/m] :  [-3577.64863599   -23.21856627   744.45289042]
Force on dipole [N]  :  [ -3.51100142e-15  -4.19215243e-16  -7.07250096e-16]

    MS Postion [um]  :  [ 13.9       -20.6675507 -15.23     ]
        Efield [V/m] :  [-3580.5273335    -21.80589117   743.37904127]
Force on dipol

    MS Postion [um]  :  [ 13.9         -7.16175383 -15.23      ]
        Efield [V/m] :  [-3660.08994748   -31.45066726   751.55470045]
Force on dipole [N]  :  [ -2.88975409e-15  -2.58826762e-17  -1.03507200e-15]

    MS Postion [um]  :  [ 13.9         -6.78484397 -15.23      ]
        Efield [V/m] :  [-3649.8194626    -34.57746175   753.37017581]
Force on dipole [N]  :  [ -2.84315345e-15  -6.35215434e-17  -1.00386224e-15]

    MS Postion [um]  :  [ 13.9         -6.40790732 -15.23      ]
        Efield [V/m] :  [-3645.65918719   -34.13450409   751.87573399]
Force on dipole [N]  :  [ -2.84315345e-15  -6.35215434e-17  -1.00386224e-15]

    MS Postion [um]  :  [ 13.9         -6.03094924 -15.23      ]
        Efield [V/m] :  [-3641.49867525   -33.69152124   750.3812072 ]
Force on dipole [N]  :  [ -2.84315345e-15  -6.35215434e-17  -1.00386224e-15]

    MS Postion [um]  :  [ 13.9         -5.65397509 -15.23      ]
        Efield [V/m] :  [-3637.33798591   -33.24851951   748.88661669]
Force on

In [85]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(500), force_z1)
ax.scatter(range(500), force_z4[::10])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(3,13)

shaking378_template = np.array(test_shaking378[1])
# shaking378_template = force_z3

fft_angles = np.angle(np.fft.rfft(shaking378_template))
fft_fft = np.abs(np.fft.rfft(shaking378_template)) * 2 / np.sqrt(5000 * 5000)
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
# fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
# tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

gravity_phases = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    gravity_phases.append(phase_all)

gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
# gravity_phases = np.vstack(gravity_phases).T

In [28]:
mm_gravity2_2 = get_alpha(0)

MLEs:  <ValueView of Minuit at 7f1998d67508>
  A: -11.896177522474318
  phi: 0.0
  sigma: 930.7935119307609


In [114]:
mm_gravity.fval, mm_gravity2.fval, mm_gravity2_2.fval

(129.02309291499253, 109.20476540830566, 109.22296265489145)

### Test new fitter

In [109]:
ll = likelihood_analyser.LikelihoodAnalyser()
def get_edm(i):

    data_z2 = np.mean(test_phases[i:(i+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'error_A': 0.1, 'limit_A': [-10, 10],
                  'A2': 0.0, 'error_A2': 0.1, 'limit_A2': [-10, 10],
                  'A3': 0.0, 'error_A3': 0.1, 'limit_A3': [-10, 10],
                  'A4': 0.0, 'error_A4': 0.1, 'limit_A4': [-100, 100],
                  'sigma':840, 'error_sigma': 10, 'limit_sigma': [0, 1000000],
               'errordef': 1, 'print_level': 0, 
                  'fix_A': True, 'fix_A2': True, 'fix_A3': True, 'fix_A4': False, 'fix_sigma': False}
    
#     fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
#               'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
#               'limit_phi': [0, np.pi],
#               'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
#               'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
    noises_std = func_sideband(data_z2, gravity_freqs_all)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_edm_multiHarmoincs2(x=data_z2, amps1=edm_amp4, phases1=edm_phases4, 
                                         amps2=edm_amp5, phases2=edm_phases5, 
                                         amps3=edm_amp6, phases3=edm_phases6, 
                                         amps4=gravity_amp, phases4=gravity_phases, scales=gravity_scales,
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
        
    return mm_tmp.values[0], mm_tmp.values[1], mm_tmp.values[2], mm_tmp.values[3], mm_tmp.fval

In [47]:
len(gravity_amp), len(edm_amp1), len(edm_amp2), len(edm_amp3)

(11, 11, 11, 11)

In [356]:
index = np.random.choice(11, 11, replace=False)
edm_amp4 = [edm_amp[i] for i in index]
index = np.random.choice(11, 11, replace=False)
edm_phases4 = [np.random.uniform()*2*np.pi for i in index]

In [93]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(1,13)[1:]

edm_template = np.array(test_shaking378[1])
edm_template = force_z6

fft_angles = np.angle(np.fft.rfft(edm_template))
fft_fft = np.abs(np.fft.rfft(edm_template)) * 2 / np.sqrt(5000 * 5000)
freq1 = np.fft.rfftfreq(len(edm_template), d=1./5000)
# fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
# tf_z = BeadDataFile.BeadDataFile(fname)
# fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
# freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
# fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
# freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

edm_phases6 = []
edm_amp6 = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    edm_phases6.append(phase_all)
    edm_amp6.append(fft_fft[freq1==drive_freq])
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
# gravity_phases = np.vstack(gravity_phases).T

In [73]:
i=100
get_edm(i), get_alpha(i)

MLEs:  <ValueView of Minuit at 277371d8>
  A: 0.0
  A2: 0.0
  A3: 0.0
  A4: -0.5376573169422443
  sigma: 840.0
MLEs:  <ValueView of Minuit at 27748068>
  A: -0.6956894578033825
  phi: 0.0
  sigma: 968.0024306905066


((0.0, 0.0, 0.0, -0.5376573169422443, 162.73958691646664), -0.6956894578033825)

In [110]:
mm_all_signal = np.array(Parallel(n_jobs=46)(delayed(get_edm)(i) for i in tqdm(range(10000))))

100%|██████████| 10000/10000 [02:26<00:00, 68.16it/s]


In [117]:
edm1_all = [mm_.values[0] for mm_ in mm_all_only1]
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(len(edm1_all)), edm1_all)
ax.scatter(range(len(mm_all_only2)), mm_all_only2)
ax.scatter(range(len(mm_all_only3)), mm_all_only3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [155]:
# edm1_all2 = [mm_.values[0] for mm_ in mm_all_only1 if (mm_.values[0]<2)&(mm_.values[0]>-2)]
# edm2_all2 = [mm_ for mm_ in mm_all_only2 if (mm_<2)&(mm_>-2)]
# edm3_all2 = [mm_ for mm_ in mm_all_only3 if (mm_<2)&(mm_>-2)]
fig,ax = plt.subplots(1,2,figsize=(9.2,4))
_ = ax[0].hist(mm_all_edm_signal[:,0], bins=50, range=(-2,2), label=r'$p_x$ scale')
_ = ax[0].hist(mm_all_edm_signal[:,1], bins=20, alpha=0.7, range=(-2,2), label=r'$p_y$ scale')
_ = ax[0].hist(mm_all_edm_signal[:,2], bins=50, alpha=0.4, range=(-2,2), label=r'$p_z$ scale')
ax[0].set(xlabel='simulation scale factor')
ax[0].legend()
_ = ax[1].hist(np.sqrt(mm_all_edm_signal[:,0]**2+mm_all_edm_signal[:,1]**2+mm_all_edm_signal[:,2]**2), alpha=0.4, range=(0,5), bins=40, 
               label=r'$\sqrt{p_x^2+p_y^2+p_z^2}$')
ax[1].set(xlabel=r'simulation scale factor')
ax[1].legend()

print(np.mean(mm_all_edm[:,0]), np.std(mm_all_edm[:,0]))
print(np.mean(mm_all_edm[:,1]), np.std(mm_all_edm[:,1]))
print(np.mean(mm_all_edm[:,2]), np.std(mm_all_edm[:,2]))
print(np.mean(np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2)))
print(np.std(np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2)))
fig.savefig("edm_only_distribution.png", format='png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.00515580522228 0.169848762275
-0.112214446385 0.578260625282
-0.0110523905528 0.089817040126
0.518619850093
0.33922754049


AttributeError: __delete__

In [98]:
_,ax = plt.subplots(figsize=(9.2,4))
titles = [r'$p_x$ scale',r'$p_y$ scale',r'$p_z$ scale']
[ax.scatter(range(10000), mm_all_edm[:,i],label=titles[i]) for i in range(3)]
ax.legend()
ax.set(ylabel='simulation scale', xlabel='dataset')
# ax.scatter(range(10000), np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2))
# ax.scatter(range(1000), mm_all_edm3[:,3])
# ax.scatter(range(10000), mm_all_edm[:,4])
# ax.scatter(range(10000), mm_all_edm5[:,4])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0, 0.5, 'simulation scale'), Text(0.5, 0, 'dataset')]

In [184]:
mm_all_edm4.shape

(10000, 5)

In [359]:
alpha_edm = [alpha_ for alpha_,chi2 in zip(mm_all_edm[:,3],mm_all_edm[:,4]) if chi2<180]
alpha_edm5 = [alpha_ for alpha_,chi2 in zip(mm_all_edm5[:,3],mm_all_edm5[:,4]) if chi2<170]
alpha_edm11 = [alpha_ for alpha_,chi2 in zip(mm_all_edm11[:,3],mm_all_edm11[:,4]) if chi2<170]

In [360]:
len(alpha_edm11)

9705

In [527]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(len(alpha_edm)), alpha_edm, label='with edm')
ax.scatter(range(len(alpha_edm5)), alpha_edm5, alpha=0.7, label='w/o edm')
ax.scatter(range(len(alpha_edm11)), alpha_edm11, alpha=0.4, label='random edm')
ax.legend()
ax.set(ylabel='simulation scale', xlabel='dataset')
print('with edm (mean, ste): ',np.mean(mm_all_edm[:,3]), np.std(mm_all_edm[:,3])/np.sqrt(10000))
print('w/o edm (mean, ste): ',np.mean(mm_all_edm5[:,3]), np.std(mm_all_edm5[:,3])/np.sqrt(10000))
print('random edm (mean, ste): ',np.mean(mm_all_edm11[:,3]), np.std(mm_all_edm11[:,3])/np.sqrt(10000))
# print('with edm (mean, ste): ', np.mean(alpha_edm), np.std(alpha_edm)/np.sqrt(10000))
# print(np.mean(alpha_edm5), np.std(alpha_edm5)/np.sqrt(10000))
# print(np.mean(alpha_edm11), np.std(alpha_edm11)/np.sqrt(10000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

with edm (mean, ste):  0.314726936746 0.147261649134
w/o edm (mean, ste):  0.830508937732 0.112488776049
random edm (mean, ste):  0.864447292648 0.148614968952


In [162]:
fig,ax = plt.subplots()
# ax.hist(mm_all_edm11[:,3], range=(-25,25), bins=30, alpha=0.7, label='random edm')
ax.hist(mm_all_edm_signal[:,3], range=(-25,25), bins=30, alpha=0.5, label='with edm')
ax.hist(mm_all_signal[:,3], range=(-25,25), bins=30, alpha=0.3, label='no edm')

ax.legend()
ax.set(xlabel=r'$\alpha \, [10^{8}]$')

# from scipy.stats import norm
# # Fit a normal distribution to the data:
# mu, std = norm.fit(mm_all_edm_signal[:,3])
# x = np.linspace(-25, 25, 100)
# p = norm.pdf(x, mu, std)
# ax.plot(x, p, '--b', linewidth=2, label='with edm')

# mu2, std2 = norm.fit(mm_all_signal[:,3])
# p = norm.pdf(x, mu2, std2)
# ax.plot(x, p, '--r', linewidth=2, label='no edm')
# ax.legend()
fig.savefig("alpha_distribution2.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### PL

In [495]:
index = np.random.choice(10000, 10000, replace=False)

In [148]:
ll = likelihood_analyser.LikelihoodAnalyser()

def calc_PL(i):

#     ii = index[i]
    ii = i
    data_z2 = np.mean(test_phases[ii:(ii+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'error_A': 0.1, 'limit_A': [-10, 10],
                  'A2': 0.0, 'error_A2': 0.1, 'limit_A2': [-10, 10],
                  'A3': 0.0, 'error_A3': 0.1, 'limit_A3': [-10, 10],
                  'A4': 0.0, 'error_A4': 0.1, 'limit_A4': [-100, 100],
                  'sigma':13.85, 'error_sigma': 10, 'limit_sigma': [0, 1000000],
               'errordef': 1, 'print_level': 0, 
                  'fix_A': True, 'fix_A2': True, 'fix_A3': True, 'fix_A4': True}
    
    noises_std = func_sideband(data_z2, gravity_freqs_all)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_edm_multiHarmoincs2(x=data_z2, amps1=gravity_amp, phases1=gravity_phases, 
                                         amps2=gravity_amp, phases2=gravity_phases, 
                                         amps3=gravity_amp, phases3=gravity_phases, 
                                         amps4=gravity_amp, phases4=gravity_phases, scales=gravity_scales,
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    

    fit_kwargs['A'] = mm_tmp.values[0]
    fit_kwargs['A2'] = mm_tmp.values[1]
    fit_kwargs['A3'] = mm_tmp.values[2]
    fit_kwargs['sigma'] = mm_tmp.values[4]
    fit_kwargs['fix_sigma'] = False
    
    PL_array = A_array*0
    PL_array = 2*10*ll.get_PL_edm_multiHarmonics(A_array=A_array, **fit_kwargs)
        
    return PL_array

In [150]:
A_array = np.arange(-0.5,0.5,0.005)
PLs = []
for i in range(1):
    print('****************************')
    print(i)
    print('****************************')
    PLs.append(np.array(Parallel(n_jobs=40)(delayed(calc_PL)(i) for i in tqdm(range(100)))))

  0%|          | 0/100 [00:00<?, ?it/s]

****************************
0
****************************


100%|██████████| 100/100 [00:50<00:00,  1.99it/s]


KeyboardInterrupt: 

In [585]:
PLs_summed_harmonics = []
PLscopy = copy.deepcopy(PLs)
PLs_summed_harmonics = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_harmonics += pp_
PLs_summed_harmonics -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_harmonics))

PLs_summed_harmonics2 = []
PLscopy = copy.deepcopy(PLs2)
PLs_summed_harmonics2 = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_harmonics2 += pp_
PLs_summed_harmonics2 -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_harmonics2))

In [586]:
0.43/np.sqrt(10)

0.13597793938724029

In [587]:
_,ax = plt.subplots()
# i=10
# [ax.plot(A_array, PLs[i], '*-') for i in range(100)]
ax.plot(A_array, PLs_summed_harmonics, '*-', label='with edm')
ax.plot(A_array, PLs_summed_harmonics2, '*-', label='no edm')
ax.legend()
ax.plot(A_array, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(-2,2), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood', title='files: 5000-6000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0),
 (-2.0, 2.0),
 Text(0.5, 0, '$\\alpha \\, [10^{8}]$'),
 Text(0, 0.5, 'Profile-Likelihood'),
 Text(0.5, 1.0, 'files: 5000-6000')]

In [588]:
type(PLs)

numpy.ndarray

In [621]:
PLlist = []
for i in range(10):
    PLlist += list(PLs_all_noEDM[i])

PLs_summed_noedm = []
PLscopy = copy.deepcopy(np.array(PLlist))
PLs_summed_noedm = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_noedm += pp_
PLs_summed_noedm -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_noedm))

PLlist2 = []
for i in range(10):
    PLlist2 += list(PLs_all_EDM[i])

PLs_summed_edm = []
PLscopy = copy.deepcopy(np.array(PLlist2))
PLs_summed_edm = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_edm += pp_
PLs_summed_edm -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_edm))


In [628]:
_,ax = plt.subplots()
ax.plot(A_array*10, PLs_summed_edm, '*-', label='with edm')
ax.plot(A_array*10, PLs_summed_noedm, '*-', label='no edm')
ax.legend()
ax.plot(A_array*10, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(-8,8), xlabel=r'$\alpha \, [10^{7}]$', ylabel = 'Profile-Likelihood', title='all files')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0),
 (-8.0, 8.0),
 Text(0.5, 0, '$\\alpha \\, [10^{7}]$'),
 Text(0, 0.5, 'Profile-Likelihood'),
 Text(0.5, 1.0, 'all files')]

In [165]:
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = fft_norm(5000, 5000)

_,ax = plt.subplots()
fft_x = np.abs(np.fft.rfft(force_x4))*scale
fft_z = np.abs(np.fft.rfft(force_z4))*scale

freqs_gravity = 3*np.arange(14)
z_to_x = np.array([fft_z[freq1==freq_]/fft_x[freq1==freq_] for freq_ in freqs_gravity])

freq1 = np.fft.rfftfreq(len(force_x4), d=1./5000)
ax.semilogy(freq1, fft_x, label='x-backgrounds')
ax.semilogy(freq1, fft_z, label='z-backgrounds')
ax.set(xlim=(1,70), ylim=(1e-19,1e-14), xlabel='frequency [Hz]', ylabel=r'ASD [N/$\sqrt{Hz.}$]')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …